In [9]:
#BEST SOLUTION EVER

# import dependencies
import os
import requests
import json
import pandas as pd
import plotly.express as px
from config import api_key
from config import mapbox_token

%reload_ext lab_black

px.set_mapbox_access_token(mapbox_token)

In [38]:
# testing out the eBird API
regionCode = "US"

url = f"https://api.ebird.org/v2/data/obs/{regionCode}/recent"

r = requests.get(url, params={"key": api_key, "back": 1})

In [39]:
r.url

'https://api.ebird.org/v2/data/obs/US/recent?key=d56mdhi9b35r&back=1'

In [40]:
r.status_code

200

In [41]:
data = r.json()

In [42]:
df = pd.DataFrame(data)
df.head()

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,brdowl,Barred Owl,Strix varia,L12327461,"White Mountain National Forest, Glen US-NH 44....",2020-09-20 05:49,1.0,44.127050,-71.209310,True,False,True,S73815106
1,amerob,American Robin,Turdus migratorius,L12327460,"129 Kings Hwy N, Westport US-CT 41.14735, -73....",2020-09-20 05:48,1.0,41.147350,-73.367013,True,False,True,S73815102
2,easowl1,Eastern Screech-Owl,Megascops asio,L12327460,"129 Kings Hwy N, Westport US-CT 41.14735, -73....",2020-09-20 05:48,2.0,41.147350,-73.367013,True,False,True,S73815102
3,passer1,passerine sp.,Passeriformes sp.,L7682942,My house,2020-09-20 05:31,2.0,26.279226,-81.826981,True,False,True,S73814950
4,grnher,Green Heron,Butorides virescens,L7682942,My house,2020-09-20 05:31,2.0,26.279226,-81.826981,True,False,True,S73814950


In [49]:
# getting a list of sub-region codes (subnational1 for US is a list of states, subnational2 for US-TX is a list of cities)
regionType = "subnational2"
parentRegionCode = "US-TX"

url = (
    "https://api.ebird.org/v2/ref/region/list/"
    + regionType
    + "/"
    + parentRegionCode
    + ".json"
)

r = requests.get(url, params={"key": api_key})

In [50]:
r.url

'https://api.ebird.org/v2/ref/region/list/subnational2/US-TX.json?key=d56mdhi9b35r'

In [51]:
r.status_code

200

In [52]:
df = pd.DataFrame(r.json())
df

,code,name
0,US-TX-001,Anderson
1,US-TX-003,Andrews
2,US-TX-005,Angelina
3,US-TX-007,Aransas
4,US-TX-009,Archer
...,...,...
249,US-TX-499,Wood
250,US-TX-501,Yoakum
251,US-TX-503,Young
252,US-TX-505,Zapata


In [56]:
# find number of species logged in different Texas cities for the last week

# first make the list of Texas cities to search
# city-level region codes for Texas are stored in df["code"] 
df["code"].head()

0    US-TX-001
1    US-TX-003
2    US-TX-005
3    US-TX-007
4    US-TX-009
Name: code, dtype: object

In [57]:
# make them into a list to iterate over for later
cities = df["code"].to_list()

# here's the first city in the list:
cities[0]

'US-TX-001'

In [74]:
# for a city, find the number of bird species observed in the last 7 days
regionCode = cities[3]
num_days = 7

url = "https://api.ebird.org/v2/data/obs/" + regionCode + "/recent"

df_city = pd.DataFrame(
    requests.get(url, params={"key": api_key, "back": num_days}).json()
)

df_city.head()

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,grhowl,Great Horned Owl,Bubo virginianus,L1449722,Home,2020-09-19 18:34,2,28.021218,-97.105625,True,False,True,S73803618
1,leafly,Least Flycatcher,Empidonax minimus,L566809,Aransas Woods (CTC 047),2020-09-19 16:04,1,27.987495,-97.109731,True,False,False,S73796866
2,grbher3,Great Blue Heron,Ardea herodias,L566809,Aransas Woods (CTC 047),2020-09-19 16:04,1,27.987495,-97.109731,True,False,False,S73796866
3,osprey,Osprey,Pandion haliaetus,L566809,Aransas Woods (CTC 047),2020-09-19 16:04,1,27.987495,-97.109731,True,False,False,S73796866
4,pibgre,Pied-billed Grebe,Podilymbus podiceps,L566809,Aransas Woods (CTC 047),2020-09-19 15:59,1,27.987495,-97.109731,True,False,False,S73796854


In [75]:
# number of bird species observed is done by counting unique speciesCode
df_city["speciesCode"].nunique()

35

In [76]:
# can do this all at once for a given city:
pd.DataFrame(requests.get(url, params={"key": api_key, "back": num_days}).json())[
    "speciesCode"
].nunique()

35

In [80]:
# now iterate over all the cities in the cities list to get a count of species observed in Texas cities for the last 7 days
num_days = 7

num_observations = []
no_observations = []

for city in cities:
    url = "https://api.ebird.org/v2/data/obs/" + city + "/recent"
    try:
        num_observations.append(
            pd.DataFrame(
                requests.get(url, params={"key": api_key, "back": num_days}).json()
            )["speciesCode"].nunique()
        )
    except Exception as e:
        no_observations.append(city)
        num_observations.append(0)

In [84]:
city_observations_df = pd.DataFrame(num_observations, index=cities).rename(
    columns={0: "observations"}
)

city_observations_df

,observations
US-TX-001,0
US-TX-003,9
US-TX-005,56
US-TX-007,35
US-TX-009,22
...,...
US-TX-499,54
US-TX-501,74
US-TX-503,16
US-TX-505,20


In [79]:
no_observations

['US-TX-001',
 'US-TX-033',
 'US-TX-059',
 'US-TX-063',
 'US-TX-083',
 'US-TX-095',
 'US-TX-125',
 'US-TX-133',
 'US-TX-137',
 'US-TX-143',
 'US-TX-147',
 'US-TX-153',
 'US-TX-155',
 'US-TX-165',
 'US-TX-193',
 'US-TX-227',
 'US-TX-235',
 'US-TX-237',
 'US-TX-247',
 'US-TX-249',
 'US-TX-263',
 'US-TX-269',
 'US-TX-295',
 'US-TX-335',
 'US-TX-345',
 'US-TX-379',
 'US-TX-383',
 'US-TX-393',
 'US-TX-403',
 'US-TX-413',
 'US-TX-417',
 'US-TX-429',
 'US-TX-433',
 'US-TX-455',
 'US-TX-457',
 'US-TX-459',
 'US-TX-483',
 'US-TX-507']

## MORE THOUGHTS:
* can keep going... merge the city name back into the new dataframe
* use request to get open weather map data for the cities (if possible going back 7 days ... may need a different weather API to see past weather data)
* could compare number of observations to temp / weather type (sunny, windy, rainy)
* use Google geocoding API to map cities to lat/lon and put number of sightings on map using plotly
* make a bar chart to show which species observed most often across the state
* probably need to actually store the results as they are requested instead of just extracting out individual pieces of info
* eBirds also has "notable" observations request which returns rare birds or unusual observations - could be fun to request a list of "notable" sightings for Austin for the last X days
* needs to be more robust generally, but this is the skeleton of the idea